### Task 1. Imports and loading
Import the packages that you've learned are needed for building linear regression models.

In [ ]:
# Imports
### YOUR CODE HERE ###



In [ ]:
# RUN THIS CELL TO IMPORT YOUR DATA. 
### YOUR CODE HERE ###

# Read in data from NYC TLC dataset provided and load into dataframe
df0=pd.read_csv("2017_Yellow_Taxi_Trip_Data.csv") # index_col parameter specified to avoid "Unnamed: 0" column when reading in data from csv

# Display "Data loaded" once the data has been loaded

Start with `.shape` and `.info()`.

In [ ]:
# Start with `.shape` and `.info()`
### YOUR CODE HERE ###



Use `.head()`.

In [ ]:
# Use `.head()`
### YOUR CODE HERE ###



Create `trip_duration`.

In [ ]:
# Create `trip_duration`
### YOUR CODE HERE ###



Check for missing data and duplicates using `.isna()` and `.drop_duplicates()`.

In [ ]:
# Check for missing data and duplicates using .isna() and .drop_duplicates()
### YOUR CODE HERE ###



Use `.describe()`.

In [ ]:
# Use .describe()
### YOUR CODE HERE ###



Create a scatterplot to visualize the relationship between variables of interest.

In [ ]:
# Create a scatterplot to visualize the relationship between variables of interest
### YOUR CODE HERE ###



Create a pairplot to visualize pairwise relationships between relevant variables.

In [ ]:
# Create a pairplot to visualize pairwise relationships between variables in the data
### YOUR CODE HERE ###



### Task 2b. Address any outliers

Use a boxplot to visualize any outliers.

In [ ]:
# Create boxplot to visualize the outliers
### YOUR CODE HERE ###



Use a boxplot to visualize the distribution of the data without outliers.

In [ ]:
# Create boxplot to visualize distribution of data without outliers
### YOUR CODE HERE ###



Remove outliers as needed.

In [ ]:
# Remove outliers as needed
### YOUR CODE HERE ###



### Task 2c. Identify correlations

Next, code a correlation matrix to help determine most correlated variables.

In [ ]:
# Correlation matrix to help determine most correlated variables
### YOUR CODE HERE ###



Visualize a correlation heatmap of the data.

In [ ]:
# Create correlation heatmap
### YOUR CODE HERE ###



### Task 3a. Select outcome variable and features

Set your Y and X variables. Y represents the outcome variable, and X represents the features.

In [ ]:
# Set your Y and X variables
### YOUR CODE HERE ###



### Task 3b. Pre-process data

To help with processing time, consider dropping irrelevant and redundant columns.

In [ ]:
# Pre-process data
### YOUR CODE HERE ###



Use `StandardScaler()` and `fit_transform()` to standardize the X variables.

In [ ]:
# Standardize the X variables
### YOUR CODE HERE ###



### Task 3c. Build model

Create training and testing sets.

In [ ]:
# Create training and testing sets
#### YOUR CODE HERE ####


Build and fit your model to the data.

In [ ]:
# Build and fit your model to the data
### YOUR CODE HERE ###



### Task 3d. Evaluate model

Evaluate your model performance by calculating the residual sum of squares and the explained variance score (R^2). Calculate the Mean Absolute Error, Mean Squared Error, and the Root Mean Squared Error.

In [ ]:
# Evaluate the model performance on the training data
### YOUR CODE HERE ###



In [ ]:
# Evaluate the model performance on the testing data
### YOUR CODE HERE ###



### Task 4a. Results

If the linear regression assumptions are met, the model results can be appropriately interpreted.

Use the code cell below to get `actual`,`predicted`, and `residual` for the testing set, and store them as columns in a `results` dataframe.

In [ ]:
# Create a `results` dataframe
### YOUR CODE HERE ###



### Task 4b. Visualize model results

Create a scatterplot to visualize `predicted` over `actual`.

In [ ]:
# Create a scatterplot to visualize `predicted` over `actual`
### YOUR CODE HERE ###



Visualize the distribution of the `residuals`.

In [ ]:
# Visualize the distribution of the `residuals`
### YOUR CODE HERE ###



Create a scatterplot of `residuals` over `predicted`.

In [ ]:
# Create a scatterplot of `residuals` over `predicted`
### YOUR CODE HERE ###

